In [5]:
import sys
sys.path.append('../src')
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
import numpy as np
import tensorflow as tf
from gait.config import pd
from gait.utils import get_data_by_overlap_percent,get_overlap_data_all_sessions,  split_test_train_by_subjects, remove_invalid_data, get_overlap_data_all_sessions
from gait.training import train_model_with_stats, build_cnn_stats_small
from gait.evalution import save_history, save_test_history, save_accuracy_loss_figure, save_confusion_matrix_figure, compute_validations_predictions,compute_validations_predictions_with_stats,compute_validations_predictions_cnn_multihead_with_stats

In [6]:
def compute_resultant_acceleration(X):
    return np.sqrt(X[:,:,0] **2 + X[:,:,1] ** 2 + X[:,:,2] ** 2)

def compute_resultant_gyro(X):
    return np.sqrt(X[:,:,3] **2 + X[:,:,4] ** 2 + X[:,:,5] ** 2)

def compute_resultant_angle(X):
    return np.sqrt(X[:,:,6] **2 + X[:,:,7] ** 2 + X[:,:,8] ** 2)

def get_std_values(X, channel):
    return np.std(X[:, :, channel], axis=1)

def get_mean_values(X, channel):
    return np.mean(X[:, :, channel], axis=1)

def get_max_values(X, channel):
    return np.max(X[:, :, channel], axis=1)

def get_median_values(X, channel):
    return np.median(X[:, :, channel], axis=1)


def get_statistic_feature_all_channels(X):
    statistic_features = []
    # statistic features of resultant vectors of 128 lenght only
    for channel in range(9,12):
        statistic_features.append(get_std_values(X, channel))
        statistic_features.append(get_mean_values(X, channel))
        statistic_features.append(get_max_values(X, channel))

    return np.vstack(statistic_features).T

def load_data(OVERLAP_PERCENT):
    X, y, subjects = get_overlap_data_all_sessions(OVERLAP_PERCENT)
    # REMOVE UNWANTED SUBJECTS
    indexes = np.where(subjects == exclude_subjects)
    X = np.delete(X, indexes[0], axis=0)
    y = np.delete(y, indexes[0], axis=0)
    subjects = np.delete(subjects, indexes[0], axis=0)
    return X, y, subjects

def augument_data(X):
     # AUGMENT DATA
    resultant_acc = compute_resultant_acceleration(X)
    resultant_gyro = compute_resultant_gyro(X)
    resultant_angle = compute_resultant_angle(X)
    resultant_acc = resultant_acc.reshape(
        resultant_acc.shape[0], resultant_acc.shape[1], 1)
    resultant_gyro = resultant_gyro.reshape(
        resultant_gyro.shape[0], resultant_gyro.shape[1], 1)
    resultant_angle = resultant_angle.reshape(
        resultant_angle.shape[0], resultant_angle.shape[1], 1)
    X = np.concatenate((X, resultant_acc), axis=2)
    X = np.concatenate((X, resultant_gyro), axis=2)
    X = np.concatenate((X, resultant_angle), axis=2)
    return X
    # AUGMENT DATA END

In [7]:
OVERLAP_PERCENT = 0
# overlapPercents = [0]
# exclude_subjects = ['ddAeJA42PXvwthbW', 'nan', 'LLZjAPTyj7muHsEf',
#                     'MMuX9YIh4NTbLZLM', 'cbOZWnI7s1y8oLD4', 'PE8D53oX060qLbdX', 'xYdtS1F8tDyjEIgN', 'sUZBISq61Y7I5tqQ']
exclude_subjects = ['ddAeJA42PXvwthbW', 'nan', 'sUZBISq61Y7I5tqQ', 'LLZjAPTyj7muHsEf',
                    'cbOZWnI7s1y8oLD4', 'EUbKPOSQgjccjtvi', 'MMuX9YIh4NTbLZLM', 'NKdnm6cN3KKAx7R4'
                    'PE8D53oX060qLbdX', 'ddAeJA42PXvwthbW', 'xYdtS1F8tDyjEIgN', '19AoxD1bgrDckd2p', 'wtyNo4LYaWXrkzA7', ]

In [8]:
X, y, subjects = load_data(OVERLAP_PERCENT)

In [9]:
X = augument_data(X)
X_stats = get_statistic_feature_all_channels(X)

In [14]:
print(X.shape)
print(X_stats.shape)

(19212, 128, 12)
(19212, 9)
